In [8]:
# importing open cv and other packages
import cv2 
import numpy as np

font = cv2.FONT_HERSHEY_COMPLEX 

In [14]:
!pip install opencv-python

  Using cached opencv_python-4.5.2.54-cp38-cp38-win_amd64.whl (34.7 MB)


In [9]:
#Targer Video file
videoFile='muh.mp4'

In [10]:
camera = cv2.VideoCapture(videoFile)

#reads the first image of the video for calibration function
cap, firstFrame = camera.read() 

In [11]:
# initialize the first frame in the video stream
firstFrame = None

#declare variables and lists
cx=0
cy=0
contourarea=[]
xcent=[]
ycent=[]
xtail=[]
ytail=[]
xhead=[]
yhead=[]

In [12]:
# make the background subtractor a variable
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

In [13]:
# Iterating trough frames as long as True, Pressing q from keyboard will stop
while True:
    
    #If Video does not loads, this will break the loop
    #Unwanted errors are not welcome
    (videoLoad, oldframe) = camera.read()
    if not videoLoad:
        break
        
    #As the provided video was not stable, I had to crop unwanted moving objects from background.
    #Not recquired for stable video files
    frame = oldframe[0:512, 210:640];
        
    #background subtractor turns the image into black and white format
    fgmask = fgbg.apply(frame)
    
    #contours of white pixel are the contours of fish
    (cnts, _) = cv2.findContours(fgmask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) 
    
    #Some Preprocessing
    #----------------------------------------------------------------------------------------------------------
    #Stores contour ares
    cntarea=[0]
    #stores center of contour area
    xcntcoord=[0]
    ycntcoord=[0]
    #Stores coordintes or fish contour
    xbody=[]
    ybody=[]
    #----------------------------------------------------------------------------------------------------------
     
        
    #loop through contours
    #-----------------------------------------------------------------------------------------------------------
    for cnt in cnts :
        #100 is selected to adjust noise
        if cv2.contourArea(cnt) < 100:          
            continue    
        
        #For creating outline of Fish Contour
        approx = cv2.approxPolyDP(cnt, 0.009 * cv2.arcLength(cnt, True), True) 

        #Drawing boundary of fish contours. 
        cv2.drawContours(frame, [approx], 0, (0, 0, 255), 5) 

        #Flattening Contour coordintes array 
        n = approx.ravel() 
        i = 0
        
        for j in n : 
            if(i % 2 == 0): 
                x = n[i] 
                y = n[i + 1]
            if(i == 0):
                # First coordinate denotes Head
                xhead.append(x)
                yhead.append(y)
            if(i==(len(n)-2)):
                #Last coordinate denotes Tail
                xtail.append(x)
                ytail.append(y)                
            i = i + 1
            
            
        #if contour area is big enough, then this code runs
        cntarea.append(cv2.contourArea(cnt)) #adds contour area to list
        M = cv2.moments(cnt)                 #finds moments of contour
        cx = int(M['m10']/M['m00'])        #calculates x coordinate of contour (i.e. centroid fish)
        cy = int(M['m01']/M['m00'])        #calculates y coordinate of contour (i.e. centroid fish)
        xcntcoord.append(cx)
        ycntcoord.append(cy)
    #----------------------------------------------------------------------------------------------------------- 
    
    
    #As Fist is the biigest object, finding it  contour
    centroidIndex=cntarea.index(max(cntarea)) 
    
    #Storing coordinate of centroid
    xcent.append(xcntcoord[centroidIndex])  
    ycent.append(ycntcoord[centroidIndex])
    
    
    #############################################################################################################
    # Tracing the path
    for i in range(len(xcent)):
        cv2.circle(frame, (xcent[i], ycent[i]),2, (0, 250, 255),thickness=-1)
        
    cv2.putText(frame, "Centroid", (xcntcoord[centroidIndex], ycntcoord[centroidIndex]),font, 0.5, (255, 0, 0))

    cv2.imshow("Circle",frame)
    #############################################################################################################
    # if the `q` key is pressed, break from the loop
    key = cv2.waitKey(10)
    if key == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

In [7]:
#create numpy array with xcoord and ycoord of fish centroid
cCoords=np.array((xcent,ycent),dtype=float) 
hCoords=np.array((xhead,yhead),dtype=float)
tCoords=np.array((xtail,ytail),dtype=float)

np.savetxt("centroid_coords.txt",np.transpose(cCoords))
np.savetxt("head_coords.txt", np.transpose(hCoords))
np.savetxt("tail_coords.txt", np.transpose(tCoords))
